In [133]:
import pandas
import numpy as np
import os
import os.path
from tabulate import tabulate
from texttable import Texttable

import latextable

In [134]:
def init_df(path):
    df = pandas.read_csv(path, sep = '\t')
    df = df[df['TotalTime2ndS'] > 0]
    df['givenid'] = df['Method'] + '-' + df['LB']
    df['nopt'] = df['status'].eq('Optimal')
    df['tottime'] = df['TotalTime2ndS'] + df['TotalTime1stS']
    df['Heuptime'] = round(df['TotalTimeHeu']/df['tottime']*100,2)
    df['MP2ndSptime'] = round(df['TotalTimeMP2ndS']/df['tottime']*100,2)
    df['MP2ndSite'] = (df['TotalIte2ndS'] - df['TotalIteOptP']) - df['TotalIteHeuTrue']
    df['RecoMIPptime'] = round(df['TotalTimeRecoMIP']/df['tottime']*100,2)
    df['RecoMIPite'] = df['TotalIte2ndS']- df['TotalIteCGTrue']
    df['RecoCGptime'] = round(df['TotalTimeRecoCG']/df['tottime']*100,2)
    df['2ndStageRecoptime'] = df['TotalTime2ndS']/df['tottime']
    df.loc[(df['Method'] == 'Covering') & (df['LB'] == 'NoBound'), ['name']] = 'BHSABasic'
    df.loc[(df['Method'] == 'Covering') & (df['LB'] == 'Simple'), ['name']] = 'EHSABasicR'
    df.loc[(df['Method'] == 'Covering') & (df['LB'] == 'Strong'), ['name']] = 'EHSABasic$R{exp}$'
    df.loc[(df['Method'] == 'DoubleCovering') & (df['LB'] == 'Strong'), ['name']] = 'EHSAExtd$R{exp}$'
    df.loc[(df['Method'] == 'DoubleCovering') & (df['LB'] == 'Simple'), ['name']] = 'EHSAExtdR'
    df.loc[(df['Method'] == 'DoubleCovering') & (df['LB'] == 'NoBound'), ['name']] = 'BHSAExtd'
    df.loc[(df['Method'] == 'BendersPICEF'), ['name']] = 'BendersPICEF'
    df['OPTptime'] = 0.0
    df.loc[(df['Method'] != 'BendersPICEF') & (df['LB'] != 'NoBound'), ['OPTptime']] = df['TotalTimeOptP']/df['tottime']*100
    df = df[(df['Method'].isin(['DoubleCovering'])) & (df['LB'] != 'Simple')]
    df_full = pandas.DataFrame(df, columns = ['Ite1stS', 'TotalIte2ndS','Instance','PDP','NDD','K','L','VertexBudget','ArcBudget','givenid','Policy', 'tottime', 'nopt','MP2ndSptime','Heuptime','RecoMIPptime', 'RecoCGptime', 'MP2ndSite', 'TotalIteHeuTrue', 'RecoMIPite', 'TotalIteCGTrue', 'status', 'NumDominatedS', 'OPTptime', 'TotalIteOptP'])
    
    return df_full

In [135]:
df = init_df("./RO_results_benchmark_p1.txt")


In [136]:
df['Heuptime']

348     3.17
349     1.02
350     5.55
351     0.58
352     4.26
        ... 
2508    0.87
2509    5.37
2510    0.66
2511    3.06
2513    1.78
Name: Heuptime, Length: 717, dtype: float64

In [137]:
vals = ['Ite1stS', 'TotalIte2ndS','tottime', 'nopt','MP2ndSptime','Heuptime','RecoMIPptime', 'RecoCGptime', 'MP2ndSite', 'TotalIteHeuTrue', 'RecoMIPite', 'TotalIteCGTrue', 'NumDominatedS', 'OPTptime', 'TotalIteOptP']
df_piv = df.pivot_table(index = ['PDP', 'K', 'L', 'VertexBudget', 'ArcBudget', 'Policy'],
        columns = 'givenid',
        aggfunc={v:'mean' if v !='nopt' else 'sum' for v in vals},
        values = vals).reset_index()
df_piv.round(2)


PDP  K  L VertexBudget ArcBudget Policy               Heuptime  \
givenid                                          DoubleCovering-NoBound   
0        100  3  3            2         0   Full                   3.39   
1        100  3  3            3         0   Full                  14.93   
2        100  3  3            4         0   Full                  26.32   
3        100  3  4            2         0   Full                   0.44   
4        100  3  4            3         0   Full                   3.71   
5        100  3  4            4         0   Full                   5.40   
6        100  4  3            2         0   Full                   1.38   
7        100  4  3            3         0   Full                   8.01   
8        100  4  3            4         0   Full                  17.62   
9        100  4  4            2         0   Full                   0.29   
10       100  4  4            3         0   Full                   1.59   
11       100  4  4            4         0   Full                   4.37   

                                             Ite1stS                        \
givenid DoubleCovering-Strong DoubleCovering-NoBound DoubleCovering-Strong   
0                        1.71                   1.50                  1.47   
1                        2.39                   1.33                  1.67   
2                        2.60                   1.31                  1.70   
3                        0.27                   1.23                  1.20   
4                        0.40                   1.21                  1.67   
5                        0.47                   1.13                  1.43   
6                        0.69                   1.47                  1.57   
7                        1.81                   1.40                  1.50   
8                        2.29                   1.23                  1.67   
9                        0.21                   1.27                  1.50   
10                       0.46                   1.23                  1.50   
11                       0.58                   1.10                  1.60   

         ...         TotalIteCGTrue                        \
givenid  ... DoubleCovering-NoBound DoubleCovering-Strong   
0        ...                 237.00                154.17   
1        ...                1164.53                319.13   
2        ...                1664.62                374.27   
3        ...                 226.40                128.83   
4        ...                 756.17                315.13   
5        ...                 965.63                291.33   
6        ...                 332.53                200.17   
7        ...                1078.10                277.80   
8        ...                1696.60                421.27   
9        ...                 216.23                192.17   
10       ...                 574.27                241.37   
11       ...                 845.90                287.23   

               TotalIteHeuTrue                                 TotalIteOptP  \
givenid DoubleCovering-NoBound DoubleCovering-Strong DoubleCovering-NoBound   
0                        67.90                 62.57                    0.0   
1                       251.67                123.30                    0.0   
2                       434.34                175.80                    0.0   
3                        70.60                 56.37                    0.0   
4                       186.38                132.17                    0.0   
5                       249.80                155.50                    0.0   
6                        85.00                 81.40                    0.0   
7                       154.70                136.00                    0.0   
8                       445.77                197.63                    0.0   
9                        67.17                 81.77                    0.0   
10                      149.27                127.73          

In [138]:
df_piv.sort_index(axis=1, level=1, inplace=True, sort_remaining = True)
df_piv.columns = [''.join(col) for col in df_piv.columns.values]
df_piv.round(2)

#df2 = df2.droplevel(('Heu_ptime', 'BendersPICEF_Strong'))
#print(df2)




,ArcBudget,K,L,PDP,Policy,VertexBudget,HeuptimeDoubleCovering-NoBound,Ite1stSDoubleCovering-NoBound,MP2ndSiteDoubleCovering-NoBound,MP2ndSptimeDoubleCovering-NoBound,...,OPTptimeDoubleCovering-Strong,RecoCGptimeDoubleCovering-Strong,RecoMIPiteDoubleCovering-Strong,RecoMIPptimeDoubleCovering-Strong,TotalIte2ndSDoubleCovering-Strong,TotalIteCGTrueDoubleCovering-Strong,TotalIteHeuTrueDoubleCovering-Strong,TotalIteOptPDoubleCovering-Strong,noptDoubleCovering-Strong,tottimeDoubleCovering-Strong
0,0,3,3,100,Full,2,3.39,1.50,173.33,10.77,...,4.84,57.36,1.87,2.94,156.03,154.17,62.57,8.60,30,48.91
1,0,3,3,100,Full,3,14.93,1.33,939.10,31.89,...,23.18,44.25,3.70,3.01,322.83,319.13,123.30,93.27,29,436.87
2,0,3,3,100,Full,4,26.32,1.31,1264.34,39.55,...,36.81,35.90,5.53,2.32,379.80,374.27,175.80,125.97,26,825.24
3,0,3,4,100,Full,2,0.44,1.23,159.23,2.59,...,0.75,69.56,1.20,5.44,130.03,128.83,56.37,7.20,28,719.42
4,0,3,4,100,Full,3,3.71,1.21,582.28,7.72,...,11.11,63.41,2.97,3.57,318.10,315.13,132.17,87.90,23,1409.03
5,0,3,4,100,Full,4,5.40,1.13,727.33,9.80,...,14.07,59.50,3.77,4.51,295.10,291.33,155.50,83.10,23,1469.34
6,0,4,3,100,Full,2,1.38,1.47,251.40,5.11,...,2.05,73.33,1.97,1.27,202.13,200.17,81.40,13.23,30,264.44
7,0,4,3,100,Full,3,8.01,1.40,943.00,20.28,...,11.97,68.36,3.07,1.08,280.87,277.80,136.00,65.27,29,473.65
8,0,4,3,100,Full,4,17.62,1.23,1277.10,23.54,...,28.64,55.46,5.43,0.65,426.70,421.27,197.63,176.90,24,1099.37
9,0,4,4,100,Full,2,0.29,1.27,152.10,1.56,...,0.79,80.57,2.40,2.14,194.57,192.17,81.77,14.93,25,1243.41


In [139]:
df_piv.drop(['ArcBudget', 'PDP', 'Policy', 'NumDominatedSDoubleCovering-Strong', 'NumDominatedSDoubleCovering-NoBound'], axis=1, inplace=True)#, 
df_piv.columns

Index(['K', 'L', 'VertexBudget', 'HeuptimeDoubleCovering-NoBound',
       'Ite1stSDoubleCovering-NoBound', 'MP2ndSiteDoubleCovering-NoBound',
       'MP2ndSptimeDoubleCovering-NoBound', 'OPTptimeDoubleCovering-NoBound',
       'RecoCGptimeDoubleCovering-NoBound', 'RecoMIPiteDoubleCovering-NoBound',
       'RecoMIPptimeDoubleCovering-NoBound',
       'TotalIte2ndSDoubleCovering-NoBound',
       'TotalIteCGTrueDoubleCovering-NoBound',
       'TotalIteHeuTrueDoubleCovering-NoBound',
       'TotalIteOptPDoubleCovering-NoBound', 'noptDoubleCovering-NoBound',
       'tottimeDoubleCovering-NoBound', 'HeuptimeDoubleCovering-Strong',
       'Ite1stSDoubleCovering-Strong', 'MP2ndSiteDoubleCovering-Strong',
       'MP2ndSptimeDoubleCovering-Strong', 'OPTptimeDoubleCovering-Strong',
       'RecoCGptimeDoubleCovering-Strong', 'RecoMIPiteDoubleCovering-Strong',
       'RecoMIPptimeDoubleCovering-Strong',
       'TotalIte2ndSDoubleCovering-Strong',
       'TotalIteCGTrueDoubleCovering-Strong',
      

In [140]:
#df_piv.rename(columns = {'K':'0_K', 'L':'1_L', 'VertexBudget':'2_VertexBudget', 'noptBendersPICEF-Strong':'3_opt',
                        #'tottimeBendersPICEF-Strong':'4_total', 'MP2ndSptimeBendersPICEF-Strong':'5_ME',
                        # 'RecoMIPptimeBendersPICEF-Strong':'6_RE', 'Ite1stSBendersPICEF-Strong':'7_ite1stS',
                        #'TotalIte2ndSBendersPICEF-Strong':'8_ite2ndS', 'MP2ndSiteBendersPICEF-Strong':'9a_MP2ndSite',
                        #'RecoMIPiteBendersPICEF-Strong':'9b_RecoMIPite'}, inplace = True)
#df_piv.rename(columns = {'K':'0_K', 'L':'1_L'}, inplace = True)
#df_piv.columns = ['0_K', '1_L', '2_VertexBudget', '3.2_HeuptimeDoubleCovering-NoBound', 
       #'3.6a_Ite1stSDoubleCovering-NoBound',
       #'3.7_MP2ndSiteDoubleCovering-NoBound', '3.3_MP2ndSptimeDoubleCovering-NoBound',
       #'3.5_RecoCGptimeDoubleCovering-NoBound', '3.8_RecoMIPiteDoubleCovering-NoBound',
       #'3.4_RecoMIPptimeDoubleCovering-NoBound',
       #'3.6c_TotalIte2ndSDoubleCovering-NoBound',       
       #'3.9_TotalIteCGTrueDoubleCovering-NoBound',
       #'3.6b_TotalIteHeuTrueDoubleCovering-NoBound', '3.0_noptDoubleCovering-NoBound',
       #'3.1_tottimeDoubleCovering-NoBound', 
       #'4.2_HeuptimeDoubleCovering-Strong',
       #'4.6a_Ite1stSDoubleCovering-Strong','4.7_MP2ndSiteDoubleCovering-Strong', '4.3_MP2ndSptimeDoubleCovering-Strong',
       #'4.5_RecoCGptimeDoubleCovering-Strong', '4.8_RecoMIPiteDoubleCovering-Strong',
       #'4.4_RecoMIPptimeDoubleCovering-Strong',
       #'4.6c_TotalIte2ndSDoubleCovering-Strong',
       #'4.9_TotalIteCGTrueDoubleCovering-Strong',
       #'4.6b_TotalIteHeuTrueDoubleCovering-Strong', '4.0_noptDoubleCovering-Strong',
       #'4.1_tottimeDoubleCovering-Strong']
df_piv = df_piv[sorted(df_piv.columns)]

In [141]:
#df_piv = df_piv.drop(['RecoCGptimeBendersPICEF-Strong', 'HeuptimeBendersPICEF-Strong', 'TotalIteCGTrueBendersPICEF-Strong', 'TotalIteHeuTrueBendersPICEF-Strong', 'NumDominatedSBendersPICEF-Strong'], axis=1)
df_piv = df_piv[['K', 'L', 'VertexBudget', 'TotalIteOptPDoubleCovering-Strong', 'TotalIteOptPDoubleCovering-NoBound', 'MP2ndSiteDoubleCovering-NoBound', 'MP2ndSiteDoubleCovering-Strong', 'OPTptimeDoubleCovering-Strong']]

In [142]:
df_piv.round(2)
#df_piv#['OPTptimeDoubleCovering-Strong']

,K,L,VertexBudget,TotalIteOptPDoubleCovering-Strong,TotalIteOptPDoubleCovering-NoBound,MP2ndSiteDoubleCovering-NoBound,MP2ndSiteDoubleCovering-Strong,OPTptimeDoubleCovering-Strong
0,3,3,2,8.60,0.0,173.33,84.87,4.84
1,3,3,3,93.27,0.0,939.10,106.27,23.18
2,3,3,4,125.97,0.0,1264.34,78.03,36.81
3,3,4,2,7.20,0.0,159.23,66.47,0.75
4,3,4,3,87.90,0.0,582.28,98.03,11.11
5,3,4,4,83.10,0.0,727.33,56.50,14.07
6,4,3,2,13.23,0.0,251.40,107.50,2.05
7,4,3,3,65.27,0.0,943.00,79.60,11.97
8,4,3,4,176.90,0.0,1277.10,52.17,28.64
9,4,4,2,14.93,0.0,152.10,97.87,0.79


In [10]:
idx = pandas.Index(df_piv.columns)
table = []
li = []

for j in idx:
    #name = j.split('_')[1]
    li.append(j)
table = []
table.append(li)
df_list = df_piv.round(2).to_numpy().tolist()
for obj in df_list:
    new_obj = [int(v) for v in obj[0:3]] + obj[3:]
    table.append(obj)
table



[['0_K',
  '1_L',
  '2_VertexBudget',
  '3_opt',
  '4_total',
  '5_ME',
  '6_RE',
  '7_ite1stS',
  '8_ite2ndS',
  '9a_MP2ndSite',
  '9b_RecoMIPite'],
 [3.0, 3.0, 2.0, 26.0, 271.82, 6.64, 78.39, 8.14, 167.29, 167.29, 167.29],
 [3.0, 3.0, 3.0, 26.0, 722.85, 25.65, 66.11, 11.83, 278.7, 278.7, 278.7],
 [3.0, 3.0, 4.0, 23.0, 854.58, 36.04, 60.33, 8.0, 516.86, 516.86, 516.86],
 [3.0, 4.0, 2.0, 28.0, 135.46, 6.7, 80.54, 3.69, 126.93, 126.93, 126.93],
 [3.0, 4.0, 3.0, 24.0, 867.87, 18.64, 75.15, 5.57, 235.8, 235.8, 235.8],
 [3.0, 4.0, 4.0, 23.0, 1122.5, 24.25, 71.05, 8.23, 386.63, 386.63, 386.63],
 [4.0, 3.0, 2.0, 19.0, 1801.11, 0.73, 92.42, 1.21, 22.45, 22.45, 22.45],
 [4.0, 3.0, 3.0, 12.0, 2434.24, 2.26, 91.52, 5.07, 134.13, 134.13, 134.13],
 [4.0, 3.0, 4.0, 15.0, 2418.72, 3.73, 91.97, 1.17, 44.13, 44.13, 44.13],
 [4.0, 4.0, 2.0, 16.0, 2099.7, 0.63, 91.13, 3.27, 34.87, 34.87, 34.87],
 [4.0, 4.0, 3.0, 15.0, 2486.05, 2.31, 93.35, 2.6, 126.0, 126.0, 126.0],
 [4.0, 4.0, 4.0, 13.0, 2659.81, 2.81,

In [11]:
latex_table = Texttable()
latex_table.set_cols_align(["c"] * len(table[0]))
latex_table.set_deco(Texttable.HEADER | Texttable.VLINES)
latex_table.add_rows(table)

s = tabulate(table, headers='firstrow', tablefmt='latex')
print(s)

with open('99_table.tex', 'wt') as f:
    f.write(s)

\begin{tabular}{rrrrrrrrrrr}
\hline
   0\_K &   1\_L &   2\_VertexBudget &   3\_opt &   4\_total &   5\_ME &   6\_RE &   7\_ite1stS &   8\_ite2ndS &   9a\_MP2ndSite &   9b\_RecoMIPite \\
\hline
     3 &     3 &                2 &      26 &    271.82 &   6.64 &  78.39 &        8.14 &      167.29 &         167.29 &          167.29 \\
     3 &     3 &                3 &      26 &    722.85 &  25.65 &  66.11 &       11.83 &      278.7  &         278.7  &          278.7  \\
     3 &     3 &                4 &      23 &    854.58 &  36.04 &  60.33 &        8    &      516.86 &         516.86 &          516.86 \\
     3 &     4 &                2 &      28 &    135.46 &   6.7  &  80.54 &        3.69 &      126.93 &         126.93 &          126.93 \\
     3 &     4 &                3 &      24 &    867.87 &  18.64 &  75.15 &        5.57 &      235.8  &         235.8  &          235.8  \\
     3 &     4 &                4 &      23 &   1122.5  &  24.25 &  71.05 &        8.23 &      386.63 &   